In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/
%cd My\ Drive/
%cd NLP
%ls

/content/drive
/content/drive/My Drive
/content/drive/.shortcut-targets-by-id/1rIZhBK8iImh_zFMtVBkm5XqK4x9VxDIr/NLP
Brainstorming.ipynb   data-validation.csv   questions.gsheet
cache_dir/            Model.ipynb           rnn.ipynb
DataCollection.ipynb  outputs/              runs/
data.csv              question_generation/  tmp/
data-mc.csv           questions.csv         ValidationDataCollection.ipynb


In [3]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install sklearn --quiet
!pip install pandas --quiet
!pip install transformers datasets accelerate nvidia-ml-py3 --quiet
!python -m nltk.downloader punkt
!pip install simpletransformers --quiet
!pip3 install torch torchvision torchaudio

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation

fatal: destination path 'question_generation' already exists and is not an empty directory.
/content/drive/MyDrive/NLP/question_generation


In [4]:
from pynvml import *
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
#from pipelines import pipeline
from transformers import Trainer
from transformers import TrainingArguments
from google.colab import files

In [5]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [6]:
print_gpu_utilization()

GPU memory occupied: 0 MB.


In [7]:
data = pd.read_csv('data.csv')
data['prefix'] = 'ask_question'

data = data.rename(columns={'story': 'input_text', 'question': 'target_text'})

# using only 25% of our training data to hopefully reduce RAM usage
data = data.sample(frac = 0.005) 

eval_df = pd.read_csv('data-validation.csv')
eval_df['prefix'] = 'ask_question'

eval_df = eval_df.rename(columns={'story': 'input_text', 'question': 'target_text'})

In [9]:
eval_df.head()

index                                        target_text  \
0      0  If the rain arrived on Saturday, what day did ...   
1      1               What was Stewart doing at the beach?   
2      2                What did the bug want from Stewart?   
3      3  What was the bug that Stewart found at the beach?   
4      4                       What does Mommy do as a job?   

                                          input_text  \
0  One sunny morning, Stewart chose to go to the ...   
1  One sunny morning, Stewart chose to go to the ...   
2  One sunny morning, Stewart chose to go to the ...   
3  One sunny morning, Stewart chose to go to the ...   
4  My mommy has a really cool job! She makes cost...   

                                             answers        prefix  
0  {'A': 'Friday', 'B': 'Monday', 'C': 'Sunday', ...  ask_question  
1  {'A': 'reading a newspaper', 'B': 'looking for...  ask_question  
2  {'A': 'food', 'B': "to read Stewart's newspape...  ask_question  
3  {'A': 'beetle', 'B': 'caterpillar', 'C': 'ant'...  ask_question  
4  {'A': 'filming movies', 'B': 'making costumes ...  ask_question

In [12]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 128

input_sequences = []
output_sequences = []

for item in data['input_text']:
  input_sequences.append(item)

for item in data['target_text']:
  output_sequences.append(item)

In [49]:
# encode the inputs
task_prefix = "create questions for the passage: "

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

In [48]:
# encode the targets
target_encoding = tokenizer(
    output_sequences, padding="longest", max_length=max_target_length, truncation=True
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100
labels = torch.tensor(labels)
# labels[labels == tokenizer.pad_token_id] = -100

In [ ]:
# forward pass
outputs = model(input_ids=input_ids, labels=labels)

tcmalloc: large alloc 3710009344 bytes == 0x7f1400ddc000 @  0x7f16afe2eb6b 0x7f16afe4e379 0x7f15d66d950e 0x7f15d66cb7c2 0x7f161063d10f 0x7f161063da51 0x7f161063daa4 0x7f161063dbb2 0x7f1611205c28 0x7f161123f132 0x7f1610967141 0x7f1610953d8c 0x7f161120b147 0x7f161120b19f 0x7f161108afc7 0x7f161108b092 0x7f1611ff82c3 0x7f1611ff8b52 0x7f16110cd216 0x7f161094b48c 0x7f161094c68a 0x7f1611393e7f 0x7f161117d0a6 0x7f168bb56f13 0x593784 0x548c51 0x51566f 0x549e0e 0x4bca8a 0x59c019 0x595ef6
tcmalloc: large alloc 3710009344 bytes == 0x7f1323bb8000 @  0x7f16afe2eb6b 0x7f16afe4e379 0x7f15d66d950e 0x7f15d66cb7c2 0x7f161063c1a7 0x7f161063c449 0x7f161063c4a6 0x7f161063c599 0x7f1611205c07 0x7f161123e82f 0x7f16106827e0 0x7f1610683cbb 0x7f1610685074 0x7f1610833daf 0x7f1611220b43 0x7f1611220bf2 0x7f1610fafb39 0x7f1611f53a72 0x7f1611f541c5 0x7f1610fdbf4a 0x7f168baf606f 0x7f168baf6366 0x593a14 0x594cd3 0x531a7c 0x4d12af 0x5122db 0x549e0e 0x4bca8a 0x59c019 0x595ef6


In [47]:
print(data['target_text'][37251])

KeyError: ignored

In [ ]:
print(data['input_text'][37251])

Located in the Laurelhurst neighborhood, Seattle Children's, formerly Children's Hospital and Regional Medical Center, is the pediatric referral center for Washington, Alaska, Montana, and Idaho. The Fred Hutchinson Cancer Research Center has a campus in the Eastlake neighborhood. The University District is home to the University of Washington Medical Center which, along with Harborview, is operated by the University of Washington. Seattle is also served by a Veterans Affairs hospital on Beacon Hill, a third campus of Swedish in Ballard, and Northwest Hospital and Medical Center near Northgate Mall.


In [ ]:
test_story = data['input_text'][37251]
test_input_ids = tokenizer(test_story, return_tensors="pt").input_ids
test_question = data['target_text'][37251]
test_label = tokenizer(test_question, return_tensors="pt").input_ids
test_output = model(input_ids=test_input_ids, labels=test_label)
#print(test_output)

In [ ]:
test_output_1 = model.generate(test_input_ids)

In [ ]:
print(tokenizer.decode(test_output_1[0], skip_special_tokens=True))

Laurelhurst neighborhood. Seattle Children's is the pediatric referral center for Washington,


In [ ]:
loss = outputs.loss

In [ ]:
outputs_1 = model.generate(input_ids)

In [ ]:
print(loss)

tensor(4.6160, grad_fn=<NllLossBackward0>)


In [ ]:
from simpletransformers.t5 import T5Model

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False,
}

model = T5Model("t5", "t5-small", args=model_args)

In [ ]:
model.train_model(data, eval_data=eval_df)

  0%|          | 0/89 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/12 [00:00<?, ?it/s]

FileNotFoundError: ignored

In [ ]:
test_story = "My mommy has a really cool job! She makes costumes for movie stars. She works in the attic of our house, so I get to see her work all the time. I watch every movie I can, and try to learn all of the different costumes by heart. Mommy says if I work hard and keep up my practice, one day I'll get to to make costumes for movie stars too! That job sounds like heaven.\newline\newlineOne day I was in the attic, helping Mommy make a boot for a costume. They were covered in little beads, and mom had to sew them on. She kept dropping the needles on the ground. Then I was helping by picking them up. I was also helping by moving the lamp around so Mommy could see the boot better. \newline\newline Ouch! I said. I accidentally stuck my finger with the needle! \newline\newline My mommy looked at my finger, and gave it a kiss. Welcome to the life of a costumer!"
input_ids = tokenizer(test_story, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

ewline ewline My mommy has a really cool job!


In [ ]:
# forward pass -- EXCEEDING RAM HERE
#loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
# loss = model(input_ids=input_ids, labels=labels).loss

In [ ]:
default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}

In [ ]:
from datasets import load_dataset
#dataset = load_dataset('csv', data_files='data.csv')
dataset = load_dataset('sagnikrayc/mctest', 'mc500', split='train') 

Generating train split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset mc_test downloaded and prepared to /root/.cache/huggingface/datasets/sagnikrayc___mc_test/mc500/1.0.0/90d0767a5628921c59c5a1e8d4b25dbf398d2e0fb6300d96bd4c364a3859e93f. Subsequent calls will reuse this data.


In [ ]:
dataset.shape

(1200, 7)

In [ ]:
training_args = TrainingArguments(per_device_train_batch_size=1, gradient_accumulation_steps=4, **default_args)

trainer = Trainer(model=model, args=training_args, train_dataset=dataset)
result = trainer.train()
print_summary(result)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


IndexError: ignored

In [ ]:
# use GPU efficiently
class TaskDataset(torch.utils.data.Dataset):
  def __init__(self, inputids, attnmasks, labels):
    self.inputids = inputids
    self.attnmasks = attnmasks
    self.labels = labels

  def __getitem__(self, idx):
    item = {}
    item['input_ids'] = self.inputids[idx]
    item['attention_mask'] = self.attnmasks[idx]
    item['labels'] = self.labels[idx]
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
'''trainer = Trainer(
		model=model,
		args=training_args,
		train_dataset=train_loader,
		eval_dataset=test_loader,
		compute_metrics=compute_metrics
		)'''

# fine-tune model
# trainer.train()

'trainer = Trainer(\n\t\tmodel=model,\n\t\targs=training_args,\n\t\ttrain_dataset=train_loader,\n\t\teval_dataset=test_loader,\n\t\tcompute_metrics=compute_metrics\n\t\t)'

In [ ]:
# hyper-parameters for our model
numepochs = 5

# large number to avoid saving models
logging_steps = 10000000
save_steps = 1000000

In [ ]:
training_args = TrainingArguments(
		output_dir=' ',
		num_train_epochs=numepochs,
		warmup_steps=1000,
		weight_decay=0.01,
    per_device_train_batch_size=32,
		logging_dir=' ',
		logging_steps=logging_steps,
		save_steps=save_steps
		)

In [ ]:
data_v = pd.read_csv('data-validation.csv')

In [ ]:
data_v = data_v['story']
data_v = data_v.drop_duplicates()
data_v.shape

(50,)

In [ ]:
story_array = data_v
story_array.size

50

In [ ]:
nlp = pipeline("e2e-qg")

In [ ]:
question_array = []

for story in story_array:
  question_array = np.append(question_array, nlp(story))

In [ ]:
question_array.size
question_df = pd.DataFrame(question_array)
questions = question_df.to_csv('questions.csv')
files.download('questions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(question_df)

ValueError: ignored